# 무도 영상 특정 인물 모자이크처리

In [1]:
import cv2, dlib
import numpy as np
from PIL import ImageFont, ImageDraw, Image

#import tensorflow.keras 
#from tensorflow.keras import backend as K

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor('models/shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('models/dlib_face_recognition_resnet_model_v1.dat')

In [2]:
def find_faces(img):
    dets = detector(img, 1)
    
    if len(dets) == 0:
        return np.empty(0), np.empty(0), np.empty(0)
    
    rects, shapes = [], []
    shapes_np = np.zeros((len(dets), 68, 2), dtype=np.int)
    for k, d in enumerate(dets):
        rect = ((d.left(), d.top()), (d.right(), d.bottom()))
        rects.append(rect)

        shape = sp(img, d)
        
        # convert dlib shape to numpy array
        for i in range(0, 68):
            shapes_np[k][i] = (shape.part(i).x, shape.part(i).y)

        shapes.append(shape)
        
    return rects, shapes, shapes_np

# 랜드마크 추출
def encode_faces(img, shapes):
    face_descriptors = []
    for shape in shapes:
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        face_descriptors.append(np.array(face_descriptor))

    return np.array(face_descriptors)

In [ ]:
 '승리3':'user_img/sl3.jpg', X
           
            '승리5':'user_img/sl5.jpg', 0
            '승리6':'user_img/sl6.jpg', 0
            '승리7':'user_img/sl7.png', 0
            '승리8':'user_img/sl8.jpg',
            '승리9':'user_img/sl9.png',
            '승리10':'user_img/sl10.jpg',
            '탑1':'user_img/top1.jpg',
            '탑2':'user_img/top2.jpg',
            '탑3':'user_img/top3.jpg',
            '탑4':'user_img/top4.jpg',
            '탑5':'user_img/top5.jpg',
            '탑6':'user_img/top6.jpg',X
            '탑7':'user_img/top7.jpg',

In [16]:
img_paths = {
            '승리1':'user_img/sl1.png',
            '승리2':'user_img/sl2.jpg',
            '승리4':'user_img/sl4.jpg',
            '승리5':'user_img/sl5.jpg',
            '승리6':'user_img/sl6.jpg',
            '승리7':'user_img/sl7.png',
            '탑1':'user_img/top1.jpg',
            '탑2':'user_img/top2.jpg',
            '탑3':'user_img/top3.jpg',
            '탑4':'user_img/top4.jpg',
            '탑5':'user_img/top5.jpg',
            '탑7':'user_img/top7.jpg'
             
            }

# 인식하고 싶은 사람들의 얼굴 랜드마크 추출후 저장
descs = []

for name, img_path in img_paths.items(): 
    img = cv2.imread(img_path)
    _, img_shapes, _ = find_faces(img)
    descs.append([name, encode_faces(img, img_shapes)[0]])
    
np.save('user_img/descs.npy', descs)
print(descs)

[['승리1', array([-0.0687706 ,  0.09059251,  0.05629875, -0.05229903, -0.16482888,
       -0.01336229, -0.079988  , -0.15324445,  0.15666482, -0.15176357,
        0.23287737, -0.05949444, -0.20585562, -0.02626969, -0.02509759,
        0.17213105, -0.20052792, -0.14857794, -0.03024867, -0.00605881,
        0.07293708,  0.01789966,  0.03830779,  0.02507115, -0.14611149,
       -0.23749547, -0.07808927, -0.06168118, -0.04371834, -0.09320866,
       -0.06865397,  0.06840748, -0.15419967, -0.07718135,  0.02925453,
        0.06096196, -0.03582066, -0.03655092,  0.19146563,  0.05008797,
       -0.22650525,  0.00938428,  0.03498069,  0.25110325,  0.22933391,
        0.03892723, -0.04263238, -0.12724514,  0.15382339, -0.20828331,
        0.07335055,  0.11607252,  0.14223568,  0.02311358,  0.03389402,
       -0.1498258 , -0.01473552,  0.16825189, -0.11128644,  0.05920645,
        0.12529856, -0.05685591,  0.0033165 , -0.13047521,  0.23395158,
        0.10671607, -0.1203109 , -0.1520761 ,  0.093250

In [18]:
cam = cv2.VideoCapture("무도.mp4") # 무도 영상
cam.set(3,640) # 너비
cam.set(4,480) # 높이
font = cv2.FONT_HERSHEY_SIMPLEX # 폰트

while True:
    
    ret, img = cam.read() 
    rects, shapes, _ = find_faces(img) # 얼굴 찾기
    descriptors = encode_faces(img, shapes) # 인코딩
    
    for i, desc in enumerate(descriptors):
        x = rects[i][0][0] # 얼굴 X 좌표
        y = rects[i][0][1] # 얼굴 Y 좌표
        w = rects[i][1][1]-rects[i][0][1] # 얼굴 너비 
        h = rects[i][1][0]-rects[i][0][0] # 얼굴 높이
        
        # 추출된 랜드마크와 데이터베이스의 랜드마크들 중 제일 짧은 거리를 찾는 부분
        descs1 = sorted(descs, key=lambda x: np.linalg.norm([desc] - x[1]))
        dist = np.linalg.norm([desc] - descs1[0][1], axis=1)
        
        if dist < 0.45: # 그 거리가 0.45보다 작다면 그 사람으로 판단 
            mosaic_img = cv2.resize(img[y:y+h, x:x+w], dsize=(0, 0), fx=0.04, fy=0.04) # 축소
            mosaic_img = cv2.resize(mosaic_img, (w, h), interpolation=cv2.INTER_AREA)  # 확대
            img[y:y+h, x:x+w] = mosaic_img # 인식된 얼굴 영역 모자이크 처리 

    cv2.imshow('camera', img)
    
    k = cv2.waitKey(10) & 0xff # 'ESC' 키 누르면 종료
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()